In [1]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
sc = SparkContext.getOrCreate()

In [2]:
# Preparons un schéma pour le dataframe df_POI_SN

from pyspark.sql.functions import *
from pyspark.sql.types import *

schema2 = StructType([\
    StructField("geoname_id", IntegerType(), True),\
    StructField("POI_name", StringType(), True),\
    StructField("POI_asciiname", StringType(), True),\
    StructField("POI_alternatenames", StringType(), True),\
    StructField("POI_latitude", StringType(), True),\
    StructField("POI_longitude", StringType(), True),\
    StructField("feature_class", StringType(), True),\
    StructField("feature_code", StringType(), True),\
    StructField("country_code", StringType(), True),\
    StructField("cc2", StringType(), True),\
    StructField("admin1_code", StringType(), True),\
    StructField("admin2_code", StringType(), True),\
    StructField("admin3_code", StringType(), True),\
    StructField("admin4_code", StringType(), True),\
    StructField("population", IntegerType(), True),\
    StructField("elevation", IntegerType(), True),\
    StructField("dem", IntegerType(), True),\
    StructField("timezone", StringType(), True),\
    StructField("Modification_date", StringType(), True),\
    StructField("AdminCodes", StringType(), True)])

In [3]:
# Importons le dataset contenant les POI du Senegal
df_POI_SN = spark.read.csv("SN.txt", sep="\t", header = "true", schema = schema2)

In [4]:
#On drop les features inutiles pour nous
df_POI_SN = df_POI_SN.drop(
    "feature_class",\
    "feature_code",\
    "country_code",\
    "cc2",\
    "admin1_code",\
    "admin2_code",\
    "admin3_code",\
    "admin4_code",\
    "population",\
    "elevation",\
    "dem",\
    "timezone",\
    "Modification_date",\
    "AdminCodes")

In [5]:
df_POI_SN.show()

+----------+--------------------+--------------------+--------------------+------------+-------------+
|geoname_id|            POI_name|       POI_asciiname|  POI_alternatenames|POI_latitude|POI_longitude|
+----------+--------------------+--------------------+--------------------+------------+-------------+
|   2243938|   Vallée du Loumbol|   Vallee du Loumbol|Vallee du Loumbel...|        15.4|    -13.93333|
|   2243939|          Ziguinchor|          Ziguinchor|Basse Casamance,D...|        12.8|    -16.36667|
|   2243940|          Ziguinchor|          Ziguinchor|ZIG,Zighinkor,Zig...|    12.56801|    -16.27326|
|   2243941|          Ziguinchor|          Ziguinchor|                null|    12.58333|    -16.26667|
|   2243942|  Ziguinchor Airport|  Ziguinchor Airport|GOGG,ZIG,Ziguinch...|    12.55569|    -16.28268|
|   2243943|          Ziguinchor|          Ziguinchor|Departement de Zi...|    12.52201|    -16.21621|
|   2243944|                Yoye|                Yoye|                nul

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Preparons un schéma pour le dataframe df_SN
schema = StructType([\
    StructField("identifier", IntegerType(), True),\
    StructField("NSID", FloatType(), True),\
    StructField("user_nickname", StringType(), True),\
    StructField("date_taken", StringType(), True),\
    StructField("date_uploaded", StringType(), True),\
    StructField("device", StringType(), True),\
    StructField("title", StringType(), True),\
    StructField("description", StringType(), True),\
    StructField("user_tags", StringType(), True),\
    StructField("machine_tags", StringType(), True),\
    StructField("longitude", StringType(), True),\
    StructField("latitude", StringType(), True),\
    StructField("accuracy", StringType(), True),\
    StructField("page_url", StringType(), True),\
    StructField("download_url", StringType(), True),\
    StructField("license_name", StringType(), True),\
    StructField("license_url", StringType(), True),\
    StructField("server_id", StringType(), True),\
    StructField("farm_id", StringType(), True),\
    StructField("secret", StringType(), True),\
    StructField("secret_original", StringType(), True),\
    StructField("extension", StringType(), True),\
    StructField("marker", StringType(), True)])

ModuleNotFoundError: No module named 'pyspark'

In [7]:
#Importons notre fichier pour tester 
df_SN = spark.read.csv("fichiertest.tsv", sep="\t", header = "true",schema = schema)

# On enlève les photos ne comportant pas de géotags et de description
df_test = df_SN.filter(~df_SN["longitude"].isin("")) 
df_test = df_test.filter(~df_SN["latitude"].isin(""))
df_test = df_test.filter(~df_SN["description"].isin(""))

In [8]:
#### Sautee pour l instant
#Produit cartesien des deux dataframes
df_BIG = df_test.crossJoin(df_POI_SN)
df_BIG.printSchema()
df_BIG.count()

root
 |-- identifier: integer (nullable = true)
 |-- NSID: float (nullable = true)
 |-- user_nickname: string (nullable = true)
 |-- date_taken: string (nullable = true)
 |-- date_uploaded: string (nullable = true)
 |-- device: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- user_tags: string (nullable = true)
 |-- machine_tags: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- accuracy: string (nullable = true)
 |-- page_url: string (nullable = true)
 |-- download_url: string (nullable = true)
 |-- license_name: string (nullable = true)
 |-- license_url: string (nullable = true)
 |-- server_id: string (nullable = true)
 |-- farm_id: string (nullable = true)
 |-- secret: string (nullable = true)
 |-- secret_original: string (nullable = true)
 |-- extension: string (nullable = true)
 |-- marker: string (nullable = true)
 |-- geoname_id: integer (nullable = true)
 |-

44836936

In [9]:
df_BIG = df_BIG.drop("identifier","NSID","user_nickname","Date_taken","secret_original","Date_uploaded","Machine_tags","Accuracy","page_URL","Download_url","device","license_name","license_url", "server_id", "farm_id", "secret", "secret original", "extension","marker","geoname_id","POI_asciiname","POI_alternatenames")

In [10]:
df_BIG.show()

+----------+-----------+---------+----------+---------+--------------------+------------+-------------+
|     title|description|user_tags| longitude| latitude|            POI_name|POI_latitude|POI_longitude|
+----------+-----------+---------+----------+---------+--------------------+------------+-------------+
|IMG_2540-1|    Senegal|  senegal|-17.028808|14.509144|   Vallée du Loumbol|        15.4|    -13.93333|
|IMG_2540-1|    Senegal|  senegal|-17.028808|14.509144|          Ziguinchor|        12.8|    -16.36667|
|IMG_2540-1|    Senegal|  senegal|-17.028808|14.509144|          Ziguinchor|    12.56801|    -16.27326|
|IMG_2540-1|    Senegal|  senegal|-17.028808|14.509144|          Ziguinchor|    12.58333|    -16.26667|
|IMG_2540-1|    Senegal|  senegal|-17.028808|14.509144|  Ziguinchor Airport|    12.55569|    -16.28268|
|IMG_2540-1|    Senegal|  senegal|-17.028808|14.509144|          Ziguinchor|    12.52201|    -16.21621|
|IMG_2540-1|    Senegal|  senegal|-17.028808|14.509144|         

In [12]:
import math

def queryPOI(longitude,latitude,POI_longitude,POI_latitude,POI_name):
    longitude = float(longitude)
    latitude = float(latitude)
    POI_longitude = float(POI_longitude)
    POI_latitude = float(POI_latitude)
    radius =50
    
    ## Calculons les coordonnées de la zone à couvrir par la bounding box 
    R = 6371000 # Rayon de la Terre en mètres
    dY = 360 * radius / R # radius est le rayon de couverture de notre zone
    dX = dY * math.cos (math.radians (latitude))
    left = longitude - dX
    bottom = latitude - dY 
    right = longitude + dX
    top = latitude + dY
    Liste_POI = list()
    if((left <= POI_longitude <= right) and (bottom <= POI_latitude <= top)):
        Liste_POI.append(POI_name)
    
    return Liste_POI

In [13]:
queryPOI = udf(queryPOI, StringType())
df_BIG = df_BIG.withColumn('Result', queryPOI('Longitude', 'Latitude','POI_longitude', 'POI_latitude','POI_name'))
df_BIG = df_BIG.filter(~df_BIG["Result"].isin("[]"))

In [14]:
import time
start_time = time.time()
df_BIG.select('Longitude', 'Latitude','Result','description', 'user_tags').show()
print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+--------------------+--------------------+--------------------+
| Longitude| Latitude|              Result|         description|           user_tags|
+----------+---------+--------------------+--------------------+--------------------+
|-17.477306|14.768794|       [Île de Yoff]|Off the coast of ...|africa+liberia,se...|
| -16.49168|16.030717|       [Sor-Marméal]|         Saint Louis| 2006,africa,senegal|
| -16.48756| 16.01674|       [Saint-Louis]|  Senegal, St. Louis|louis,saint,seneg...|
| -16.48756| 16.01674|        [Eau Claire]|  Senegal, St. Louis|louis,saint,seneg...|
| -16.48756| 16.01674|       [Saint-Louis]|  Senegal, St. Louis|louis,saint,seneg...|
| -16.48756| 16.01674|        [Eau Claire]|  Senegal, St. Louis|louis,saint,seneg...|
| -16.50119|16.049772|[Marigot de Témédas]|      fleuve senegal|africa,senegal,tr...|
| -16.50119|16.049772| [Marigot de Ndiago]|      fleuve senegal|africa,senegal,tr...|
| -16.50119|16.049772|        [Bop Nquior]|      fleuv

In [15]:
feature_group = ['Longitude','Latitude','description','user_tags']
df1 = df_BIG.groupby(feature_group).agg(concat_ws(", ", collect_list(df_BIG.Result)).alias("Points of interest"))
data_joined = df_BIG.drop('POI_name','POI_latitude','POI_longitude','Result').join(df1, feature_group).dropDuplicates()

In [96]:
from fuzzywuzzy import process
def filterPOI(description, title, POI_list):
    POI_list = POI_list.strip('], [')
    POI_list = POI_list.split('], [')
    if (len(POI_list)== 1):
        return (str(POI_list))
    else:
        description_filtered = process.extract(str(description),POI_list)
        high_desc_score = list()
        for p in description_filtered:
            if (p[1] >= 80):  ## On vérifie si le score est supérieur à notre seuil 80
                high_desc_score.append(p)
            
        if (len(high_desc_score) == 1):
            return str(high_desc_score)
        
        else :
            desc_title_filtered = list()
            title_filtered = process.extract(str(title),POI_list)
            for i in description_filtered:
                for j in title_filtered:
                    if (i[0] == j[0]):
                        desc_title_filtered.append(tuple((i[0],i[1]+j[1]))) 
            
            desc_title_filtered.sort(key=lambda x:x[1],reverse = True)
            return str(desc_title_filtered)

In [97]:
# Créons le udf 
filterPOI_udf = udf(filterPOI, StringType())
data_joined = data_joined.withColumn("POI_filtered_Lev",filterPOI_udf('description','title','Points of interest'))

In [98]:
start_time = time.time()
data_joined.toPandas().to_csv("POI_filt_Lev.csv")
print("--- %s seconds ---" % (time.time() - start_time))

--- 917.9474859237671 seconds ---


In [88]:
str2Match = "The lowest shutter speed I have ever shot during a KAP session at 1/250.  Turned out well and had many keepers, but this photograph is the only one worthy of upload.%0A%0AAbout ten minutes after this shot, they turned the lights on.  There was still a bit of ambient light, so now I wonder what 1/150 or 1/100 would do while the light is on?l%0A%0A<a href=http://www.unc.edu/~rowlett/lighthouse/sen.htm"" rel=""nofollow"">www.unc.edu/~rowlett/lighthouse/sen.htm</a>%0A%0AIt has seen better days since it was built in 1864.  <a href=""http://www.leuchtturm-welt.net/HTML/AFRICAPK/ORIGINAL/MAMELLES.JPG"" rel=""nofollow"">Have no idea when this was taken.</a>%0A%0AImage was captured by a camera suspended by a kite line. Kite Aerial Photography (KAP) 7' Rok"
title = "Phare Des Mamelles"
stringPOI = "[Les Mamelles], [Phare des Mamelles]"
POI_list_test = stringPOI.split('], [')
print(process.extract(title,POI_list_test))
print(process.extract(str2Match,POI_list_test))
filterPOI(str2Match,title,stringPOI)

[('Phare des Mamelles]', 100), ('[Les Mamelles', 86)]
[('[Les Mamelles', 57), ('Phare des Mamelles]', 57)]


'Phare des Mamelles'